In [6]:
import pandas as pd
import numpy as np
import re

In [7]:
Yaqut_PlcaeTypeGeo = pd.read_excel(r"Yaqut-PlcaeTypeGeo.xlsx")

In [8]:
#get text and pattern
#return True if the pattern in the string
def check_pattern(text,pattern):
    if pattern in text:
        return True
    return False

#get Dataframe
#fill all the empty cloumns in the dataframe
#return dataFrame
def fill_nan(df):
    i = 0
    for row in df.iterrows():
        if(df['text'].iloc[i] is np.nan):
            df['text'].iloc[i] = ""
        i+=1
    return df

#get Dataframe
#check if the text for single place has one of the patterns
#return dataframe with cloumn of True/ false , True if at least the text has one of the patterns 
def get_if_there_is_pattern(df):
    check = False
    i = 0
    myPatt = ""
    rowcounter = 0
    df = fill_nan(df)
    column_names = ["Place_Name","Place_ID","text","had_the_pattern","the_pattern"]
    places_patterns = pd.DataFrame(columns = column_names)
    patterns = ["بينه وبين","بينها وبين","وبينه وبين","وبينها وبين"]
    for row in df.iterrows():
        placeName = df['placename'].iloc[i]
        placeID = df['item'].iloc[i]
        text = df['text'].iloc[i]
        for pattern in patterns:
            if check == False:
                check = check_pattern(text,pattern)
                if(check == True):
                    myPatt = pattern
        places_patterns.loc[rowcounter] = [placeName, placeID, text, check,myPatt]
        check = False
        i += 1
        myPatt = ""
        rowcounter += 1
    return places_patterns

#get DataFrame
#check if the place has pattern is true
#retutn Data frame with the places that has one of the patterns
def places_with_pattern(df):
    i = 0
    column_names = ["Place_Name","Place_ID","text","the_pattern"]
    places_patterns = pd.DataFrame(columns = column_names)
    rowcounter = 0
    for row in df.iterrows():
        placeName = df['Place_Name'].iloc[i]
        placeID = df['Place_ID'].iloc[i]
        text = df['text'].iloc[i]
        patt = df['the_pattern'].iloc[i]
        if df['had_the_pattern'].iloc[i]:
            places_patterns.loc[rowcounter] = [placeName, placeID,text,patt]
            rowcounter += 1
        i += 1
    return places_patterns

In [9]:
checkpatterns = get_if_there_is_pattern(Yaqut_PlcaeTypeGeo)
with_patterns = places_with_pattern(checkpatterns)

In [36]:
def create_conn(df):
    i = 0
    column_names = ["Place_Name","Place_ID","text","refernce_place","N","MeasuresIndex"]
    places_patterns = pd.DataFrame(columns = column_names)
    rowcounter = 0
    for row in df.iterrows():
        placeName = df['Place_Name'].iloc[i]
        placeID = df['Place_ID'].iloc[i]
        text = df['text'].iloc[i]
        pattern = df['the_pattern'].iloc[i]
        before_keyword, keyword, after_keyword = text.partition(pattern)
        words = after_keyword.split()
        if len(words) == 1:
            places_patterns.loc[rowcounter] = [placeName, placeID,text,words[0],0,"X"]
        elif len(words) == 2:
            places_patterns.loc[rowcounter] = [placeName, placeID,text,words[0],0,words[1]]
        else:
            places_patterns.loc[rowcounter] = [placeName, placeID,text,words[0],words[1],words[2]]
        i +=1
        rowcounter +=1
    return places_patterns

In [37]:
check = create_conn(with_patterns)
check

,Place_Name,Place_ID,text,refernce_place,N,MeasuresIndex
0,آبسكون,JK_000129_000006,آبسكون والنسبة الآبسكوني بفتح الهمزة وسكون الأ...,جرجان,ثلاثة,أيام
1,آبل,JK_000129_000010,آبل من قرى حمص من جهة القبلة بينها وبين حمص نح...,حمص,نحو,ميلين
2,آخر,JK_000129_000019,آخر والنسبة الآخري بضم الخاء المعجمة والراء قص...,سمنان,تسعة,فراسخ
3,آذنة,JK_000129_000021,آذنة بكسر الذال المعجمة والنون خيال من أخيلة ح...,فيد,نحو,عشرين
4,آسك,JK_000129_000033,آسك بفتح السين المهملة وكاف كلمة فارسية قال أب...,أرجان,يومان,وبينها
...,...,...,...,...,...,...
968,تاموكيدا,JK_000130_014403,بلد بالمغرب بينه وبين المسيلة مرحلتان,المسيلة,0,مرحلتان
969,دير مرجرجيس,JK_000130_014410,فوق بلد بينها وبين جزيرة ابن عمر على ثلاثة فرا...,جزيرة,ابن,عمر
970,حصن مسلمة,JK_000130_014414,بالجزيرة بين رأس عين والرقة بناه مسلمة بن عبد ...,البليخ,ميل,ونصف
971,تفتد,JK_000130_014443,بالفتح ثم السكون وتاء أخرى مفتوحة وضبطه الزمخش...,قلهى,جبل,يقال
